In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score, KFold
from sklearn.compose import ColumnTransformer, TransformedTargetRegressor
from skopt.space import Real, Integer, Categorical
from skopt import gp_minimize

import utils.dev_config as dev_conf
import utils.preprocessing as prep
import utils.optimization as opt

In [2]:
dirs = dev_conf.get_dev_directories("../dev_paths.txt")
unified_dsets = ["unified_cervical_data", "unified_uterine_data", "unified_uterine_endometrial_data"]
matrisome_list = f"{dirs.data_dir}/matrisome/matrisome_hs_masterlist.tsv"

In [3]:
i = 0

In [4]:
matrisome_df = prep.load_matrisome_df(matrisome_list)

In [5]:
seed = 123
rand = np.random.RandomState()

# Load and filter survival data

In [6]:
event_code = {"Alive": 0, "Dead": 1}
covariate_cols = ["age_at_diagnosis", "race", "ethnicity"]
dep_cols = ["figo_stage"]
cat_cols = ["race", "ethnicity"]
survival_df = prep.load_survival_df(f"{dirs.data_dir}/{unified_dsets[i]}/survival_data.tsv", event_code)

In [7]:
filtered_survival_df = (
    prep.decode_figo_stage(survival_df[["sample_name"] + dep_cols + covariate_cols].dropna(), to="n")
        .pipe(pd.get_dummies, columns=cat_cols)
        .reset_index(drop = True)
        .pipe(prep.cols_to_front, ["sample_name", "figo_num"])
)
filtered_survival_df.columns = filtered_survival_df.columns.str.replace(' ', '_')
print(filtered_survival_df.shape)
# filtered_survival_df.head()

(255, 12)


# Load normalized matrisome count data

In [8]:
norm_matrisome_counts_df = pd.read_csv(f"{dirs.data_dir}/{unified_dsets[i]}/norm_matrisome_counts.tsv", sep='\t')
norm_filtered_matrisome_counts_t_df = prep.transpose_df(
    norm_matrisome_counts_df[["geneID"] + list(filtered_survival_df.sample_name)], "geneID", "sample_name"
)
print(norm_filtered_matrisome_counts_t_df.shape)
# norm_filtered_matrisome_counts_t_df.head()

(255, 1009)


# Join survival and count data

In [9]:
joined_df = (
    pd.merge(filtered_survival_df, norm_filtered_matrisome_counts_t_df, on="sample_name")
        .set_index("sample_name")
)
print(joined_df.shape)
# joined_df.head()

(255, 1019)


# Optimize model

In [10]:
rand.seed(seed)
x_df, y_df = prep.shuffle_data(joined_df, rand)

In [11]:
def objective(h_params, X, y, r, verbose=True):
    if verbose:
        print(h_params)
    model = GradientBoostingClassifier(
        loss="deviance",
        learning_rate=h_params[0],
        n_estimators=h_params[1],
        max_depth=h_params[2],
        max_features=h_params[3],
        min_samples_split=h_params[4],
        min_samples_leaf=h_params[5],
        random_state=r
    )
    return -np.mean(cross_val_score(
        model,
        X,
        y,
        cv=KFold(n_splits=5),
        n_jobs=-1,
        scoring="f1_weighted"
    ))

In [12]:
space = [
#     Categorical(["deviance", "exponential"], name="loss"),
    Real(1e-3, 1e-1, name="learning_rate"),
    Integer(int(1e2), int(1e3), name="n_estimators"),
    Integer(2, 5, name="max_depth"),
    Categorical(["auto", "sqrt", "log2"], name="max_features"),
    Integer(int(2), int(6), name="min_samples_split"),
    Integer(int(1), int(3), name="min_samples_leaf")
]
n_initial = 10 * len(space)
n_calls = 50 * len(space)

In [13]:
callback_file = f"{unified_dsets[i]}_opt_gbc_h_params.tsv"
try:
    os.remove(callback_file)
except OSError:
    pass

res = gp_minimize(
    lambda h_ps: objective(h_ps, x_df, y_df, rand),
    space,
    verbose=True,
    random_state=rand,
    n_initial_points=n_initial,
    n_calls=n_calls,
    n_jobs=-1,
    callback=lambda x: opt.save_callback(x, callback_file, n = 5, sep="\t")
)

Iteration No: 1 started. Evaluating function at random point.
[0.00883574711308232, 873, 4, 'sqrt', 3, 1]
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 8.4085
Function value obtained: -0.4121
Current minimum: -0.4121
Iteration No: 2 started. Evaluating function at random point.
[0.014703141705377391, 459, 3, 'log2', 2, 1]
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 2.3021
Function value obtained: -0.3948
Current minimum: -0.4121
Iteration No: 3 started. Evaluating function at random point.
[0.08135279048011816, 521, 4, 'auto', 4, 3]
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 96.5418
Function value obtained: -0.4297
Current minimum: -0.4297
Iteration No: 4 started. Evaluating function at random point.
[0.058635370455441156, 285, 4, 'log2', 5, 1]
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 1.9339
Function value obtained: -0.4127
Current minimum: -0.4297
Iteration No: 5 started. Evaluating funct

/home/carcook/anaconda3/envs/TCGA-round1/lib/python3.8/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


*****redundant*****
Iteration No: 88 ended. Search finished for the next optimal point.
Time taken: 6.4806
Function value obtained: -0.5135
Current minimum: -0.5135
Iteration No: 89 started. Searching for the next optimal point.
[0.034112730272401526, 997, 5, 'auto', 4, 2]
Iteration No: 89 ended. Search finished for the next optimal point.
Time taken: 236.1989
Function value obtained: -0.4396
Current minimum: -0.5135
Iteration No: 90 started. Searching for the next optimal point.
[0.054454380439237254, 963, 2, 'log2', 6, 2]
Iteration No: 90 ended. Search finished for the next optimal point.
Time taken: 4.0789
Function value obtained: -0.4592
Current minimum: -0.5135
Iteration No: 91 started. Searching for the next optimal point.
[0.09847336234812758, 978, 2, 'log2', 5, 1]
Iteration No: 91 ended. Search finished for the next optimal point.
Time taken: 4.2209
Function value obtained: -0.4744
Current minimum: -0.5135
Iteration No: 92 started. Searching for the next optimal point.
[0.1, 10

/home/carcook/anaconda3/envs/TCGA-round1/lib/python3.8/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


*****redundant*****
Iteration No: 96 ended. Search finished for the next optimal point.
Time taken: 6.3444
Function value obtained: -0.4839
Current minimum: -0.5135
Iteration No: 97 started. Searching for the next optimal point.
[0.08861090737837704, 1000, 2, 'sqrt', 4, 1]
Iteration No: 97 ended. Search finished for the next optimal point.
Time taken: 6.3780
Function value obtained: -0.4827
Current minimum: -0.5135
Iteration No: 98 started. Searching for the next optimal point.
[0.05195600612067619, 1000, 2, 'sqrt', 6, 3]
Iteration No: 98 ended. Search finished for the next optimal point.
Time taken: 6.2910
Function value obtained: -0.4776
Current minimum: -0.5135
Iteration No: 99 started. Searching for the next optimal point.
[0.03464886589367248, 104, 5, 'auto', 2, 3]
Iteration No: 99 ended. Search finished for the next optimal point.
Time taken: 23.8445
Function value obtained: -0.4628
Current minimum: -0.5135
Iteration No: 100 started. Searching for the next optimal point.
[0.07196

...

Iteration No: 296 ended. Search finished for the next optimal point.
Time taken: 11.9473
Function value obtained: -0.4926
Current minimum: -0.5176
Iteration No: 297 started. Searching for the next optimal point.
[0.1, 1000, 2, 'sqrt', 2, 3]


/home/carcook/anaconda3/envs/TCGA-round1/lib/python3.8/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Iteration No: 297 ended. Search finished for the next optimal point.
Time taken: 12.2059
Function value obtained: -0.4895
Current minimum: -0.5176
Iteration No: 298 started. Searching for the next optimal point.
[0.1, 1000, 2, 'sqrt', 6, 3]


/home/carcook/anaconda3/envs/TCGA-round1/lib/python3.8/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Iteration No: 298 ended. Search finished for the next optimal point.
Time taken: 11.8798
Function value obtained: -0.4894
Current minimum: -0.5176
Iteration No: 299 started. Searching for the next optimal point.
[0.06559178273616784, 1000, 2, 'sqrt', 6, 3]
Iteration No: 299 ended. Search finished for the next optimal point.
Time taken: 12.2717
Function value obtained: -0.4945
Current minimum: -0.5176
Iteration No: 300 started. Searching for the next optimal point.
[0.060237331602564774, 1000, 2, 'sqrt', 4, 3]
Iteration No: 300 ended. Search finished for the next optimal point.
Time taken: 12.3686
Function value obtained: -0.4844
Current minimum: -0.5176
